# Get Error of EMS Predictions

1. Get all models 
2. Train model
3. Predict on testset
4. Calc Errors

In [1]:
#!pip install -r requirements.txt --user
#!pip install plotly
#!pip install matplotlib
#!pip install seaborn

In [2]:
# change current directory to the project directory
import os
os.chdir('/home/mrks/linux-Documents/efficientIO')


# environment setup
os.environ["CLUSTER_NAME"] = "bms"
os.environ["DB_USER"] = "TSaPh3WxR99FY8W0O7HQYs85D4JMpnFm"
os.environ["DB_PASS"] = "PQmkMtppq28myb5sevye3lvWCkmLh19O"
os.environ["DB_ADDR"] = "localhost"
os.environ["DB_PORT"] = "5432"
os.environ["DB_NAME"] = "lynus"
os.environ["DB_MODE"] = "disable"
os.environ['S3_ENDPOINT'] = "sos-at-vie-1.exo.io"
os.environ['S3_ACCESS_KEY'] = "EXO865e79270e7bf9bcebc0e587"
os.environ['S3_SECRET_KEY'] = "SBs7m9_0tUzcKI4FMouFrxkBZeCH_D2l8a82_RFMfms"
os.environ['S3_SECURE'] = "1"
os.environ["EMPA_ADDR"] = "http://127.0.0.1:9000/"
os.environ['WEATHER_API_KEY'] = ""


In [3]:
import psycopg
import pandas as pd

# Get DB connection and all EMS models

try:
    # Establish a connection to the PostgreSQL database
    connection = psycopg.connect(
        dbname=os.environ["DB_NAME"],
        user=os.environ["DB_USER"],
        password=os.environ["DB_PASS"],
        host=os.environ["DB_ADDR"],
        port=os.environ["DB_PORT"]
    )

    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    cursor.execute("SELECT models.id as model_id, models.r->'data'->>'type' as model_type, project.name as project_name, models.r->>'project_id' as project_id \
FROM models \
JOIN project ON models.r->>'project_id'= project.id::text \
WHERE models.r -> 'data' ->> 'type' LIKE '%EMS%' \
")
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Get column names
    

except psycopg.Error as e:
    print(f"Unable to connect to the database. Error: {e}")

finally:
    # Close the cursor and connection, regardless of whether the try block succeeds or raises an exception
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")

# Convert the result to a Pandas DataFrame
model_df = pd.DataFrame(result, columns=columns)
model_df

Connection closed.


,model_id,model_type,project_name,project_id
0,d9cdab3e-acd9-499b-b53c-05d698b23b95,EMS,Projekt Hammerschmied,673dfecd-87ae-41b4-adc6-bf4a9ec1d31b
1,a2281880-cb84-4705-a09d-f80367bdd342,EMS,Mayrhofer,e51c2607-26db-4106-b5ea-e880014dacba
2,0ed1bfca-b33e-4063-a907-d5ebe85bb622,EMS,Barac Michael,c4f8afbf-2040-4517-95e7-ed7f0d999242
3,4d908d4d-b57d-47e8-ab5a-738326e72be9,EMS,HAPU,b818e25e-2206-4375-bd9d-0099f6e8c3ab
4,660decfe-1fe8-4c3d-b6ea-f29d22697d94,EMS,HAPU,b818e25e-2206-4375-bd9d-0099f6e8c3ab
5,9d79ac11-63df-45ad-82a2-0075652d0506,EMS,Feigl,f2612c04-11b1-4280-957b-bcd7afeefd1d
6,99582f62-bad3-4bbb-8ff6-42d9f0827a1f,EMS,Projekt Kogler,f78d81b3-e56c-4072-b5bd-ca591893463c
7,15334478-785b-4345-8a88-cc748e9abcb7,EMS,Jodlbauer,189e4327-6e6d-4ffb-9c03-09f2cb3fc9e8
8,fd2f03d2-099b-45e3-b300-b01853e06f00,EMS,Brandmoa,32673a79-600a-4644-a67c-4e88e91f75b0
9,d1793fbb-e453-4a38-a37d-235e0111afe1,EMS,Projekt Katzenhofer,9c801c0d-8c2c-4278-a890-f6835690b001


In [48]:
import datetime
from ai.model.ems_model import EmsModel
from ai.preprocessing.ems_preprocessing import preprocess_ems_data, get_ems_features
from ai.interface.model_holder import ModelHolder
from ai.model_holder.ems_model_holder import EmsModelHolder

def get_model(model, model_id, conf):
    model_id = model_id
    model_holder_type = EmsModelHolder
    model_holder = ModelHolder.load_model_holder_from_s3(model_id, model_holder_type, model_load=False)

    end = datetime.datetime.now()
    start = datetime.datetime.fromtimestamp(model_holder.get_start())

    if (end - start) <= datetime.timedelta(days=14):
        raise ValueError("Too few data (< 14 days) for EMS training!")

    if end - start > datetime.timedelta(weeks=conf["train_duration_weeks"]):
        start = end - datetime.timedelta(weeks=conf["train_duration_weeks"])            
    end_training = end - datetime.timedelta(weeks=1)

    df = model_holder.get_dh().get_data(int(start.timestamp()), int(end_training.timestamp()))
    df = preprocess_ems_data(df, time_resolution=conf["time_resolution"], filter_data=conf["filter_data"], add_hour=conf["add_hour"])

    (x, y_consumption, y_pv, y_generator, y_grid, y_battery,
            y_charge, y_house, y_electric, y_pump, y_consumer,
            y_soc, y_tpow_battery, y_tpow_charge, y_tpow_electric, y_tpow_pump,
            y_tpow_consumer, recursive) = get_ems_features(
            df,
            lag_steps=conf["lag_steps"],
            prediction_steps=conf["prediction_steps"],
            time_resolution=conf["time_resolution"],
            lag_label=conf["lag_label"])

    # Fit model
    if isinstance(model, EmsModel):
        model.fit(x, y_consumption, y_pv, y_generator, y_grid, y_battery, y_charge,
                    y_house, y_electric, y_pump, y_consumer, y_soc, y_tpow_battery,
                    y_tpow_charge, y_tpow_electric, y_tpow_pump, y_tpow_consumer, recursive)
    else:
        model.fit(x.values,y_pv)
    
    df_test = model_holder.get_dh().get_data(int(end_training.timestamp()), int(end.timestamp()))
    df_test = preprocess_ems_data(df_test, time_resolution=conf["time_resolution"], filter_data=conf["filter_data"], add_hour=conf["add_hour"])

    (x, y_consumption, y_pv, y_generator, y_grid, y_battery,
        y_charge, y_house, y_electric, y_pump, y_consumer,
        y_soc, y_tpow_battery, y_tpow_charge, y_tpow_electric, y_tpow_pump,
        y_tpow_consumer, recursive) = get_ems_features(
        df_test,
        lag_steps=conf["lag_steps"],
        prediction_steps=conf["prediction_steps"],
        time_resolution=conf["time_resolution"],
        lag_label=conf["lag_label"])
    
    return model, x, y_pv, recursive
    

In [49]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import numpy as np

def get_errors(metrics, pairs):
    # Calc error measures
    error_dict = dict()
    for mkey, mval in metrics.items():
        model_errors = pairs.copy()
        mean = 0
        count = 0

        for key, val in model_errors.items():
            if len(val[0]) == 0:
                print("error 1")
                continue
            else:
                if key != "y_consumption":
                    error = 0
                    instance_count = 0
                    for instance in range(0, len(val[0])):
                        count+=1
                        instance_count += 1
                        er = mval["metric"](val[0][instance], val[1][instance])
                        if mkey == "Corr":
                            er = er[0,1]
                        
                        if "limit" in mval.keys():
                            if er > mval["limit"]:
                                er = mval["limit"]
                        error+=er
                        mean+=er
                    error = error/instance_count
                else:
                    count+=1
                    error = mval["metric"](val[0], val[1])
                    if mkey == "Corr":
                            error = error[0,1]
                    
                    if "limit" in mval.keys():
                        if error > mval["limit"]:
                            error = mval["limit"]
                    mean+=error

                model_errors[key] = error
        
        model_errors["mean"] = mean/count
                
        #model_errors = {key: value for key, value in model_errors.items() if value != -99}
        error_dict[mkey] = model_errors

    return error_dict


In [50]:
def get_stats(errors):
    data = [(key, value['MAPE']["y_pv"]) for key, value in errors.items()]

    # Creating a DataFrame
    df = pd.DataFrame(data, columns=['Project', 'y_pv'])
    print(df)

    print(f"max: {df['y_pv'].max()}, min: {df['y_pv'].min()}, mean: {df['y_pv'].mean()}")

    

In [51]:
import datetime
from ai.model.ems_model import EmsModel
from ai.preprocessing.ems_preprocessing import preprocess_ems_data, get_ems_features
from ai.interface.model_holder import ModelHolder
from ai.model_holder.ems_model_holder import EmsModelHolder
from sklearn.svm import SVR

# Get start and end time to create train and test set

conf={
    "train_duration_weeks": 5,
    "lag_steps":[15, 30, 45, 60],
    "prediction_steps":[15, 30, 45, 60],
    "time_resolution":15,
    "lag_label":[15],
    "filter_data":True,
    "add_hour":True,
    "model": SVR,
    "model_params": {}
}

project_errors = {}

for idx, model_id in enumerate(model_df["model_id"]):
    print(model_id)

    model, x, y_pv, recursive = get_model(conf["model"], model_id, conf)

    if isinstance(model, EmsModel):
        (y_consumption_pred, y_pv_pred, y_generator_pred, y_grid_pred, y_battery_pred, y_charge_pred,
        y_house_pred, y_electric_pred, y_pump_pred, y_consumer_pred, y_soc_pred, y_tpow_battery_pred,
        y_tpow_charge_pred, y_tpow_electric_pred, y_tpow_pump_pred, y_tpow_consumer_pred) = model.predict(x, recursive=recursive)
    else:
        y_pv_pred = model.predict(x.values)
    

    # prepare error calcs
    pairs = {
        "y_pv":(y_pv,y_pv_pred),
        }
        
    metrics = {
        "MAPE": {"metric": mean_absolute_percentage_error, "limit": 1},
        "Corr": {"metric": np.corrcoef},
        "MAE": {"metric": mean_absolute_error},
        # "R2": {"metric": r2_score} prone to outliers
        }
    
    project_errors[model_df.iloc[idx]["project_name"]] = get_errors(metrics, pairs)

get_stats(project_errors)




d9cdab3e-acd9-499b-b53c-05d698b23b95


AttributeError: 'DataFrame' object has no attribute '_validate_params'

True